In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
with open("2018-02-14_focus/2018-02-14_focus_group.json") as focus_group:
    focus = json.load(focus_group)

In [3]:
with open("krx_code.json") as krx:
    krx_code = json.load(krx)

In [4]:
with open("2018-02-14_opening_increase.json") as oi:
    increase = json.load(oi)

In [ ]:
import bs4
import urllib3
import re

def opening_increase(code, date):

    #code: string
    #date: string, e.g. 2018-02-14

    target = "http://finance.naver.com/item/sise_time.nhn?code=" + code + "&thistime=" + re.sub('[-]', '', date) + "090001&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    
    data_list = soup.find_all("tr")[2].find_all("td",class_="num")
    price = data_list[0].text
    
    return price != '\xa0' and "상승" in str(soup)
    
#json = json.dumps(rv, ensure_ascii=False)
#f = open("2018-02-14_opening_increase.json","w")
#f.write(json)
#f.close()

In [ ]:
'''
rv = []
c = 0
for stock in focus:
    c+= 1
    if opening_increase(krx_code[stock], "2018-02-14"):
        rv.append(stock)
    print(c)
'''

In [ ]:
'''
json = json.dumps(rv, ensure_ascii=False)
f = open("2018-02-14_opening_increase.json","w")
f.write(json)
f.close()
'''

In [ ]:
'''
def ten_minutes_before(time):
    
    if time[-2:] == "00":
        prefix = '0' + str(int(time[11:13]) - 1) + ":50" if int(time[11:13]) <= 10 \
        else str(int(time[11:13]) - 1) + ":50"
        time = time[:-5] + prefix
    
    else:
        time = time[:-2] + str(int(time[-2:]) - 10)
    
    return time
'''

In [5]:
import bs4
import urllib3

def scrape_price_history(code, time):
    '''
    code: string
    time: string, e.g. 201802090910
    '''

    target = "http://finance.naver.com/item/sise_time.nhn?code=" + code + "&thistime=" + time + "01&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    data_list = soup.find_all("tr")[2].find_all("td",class_="num")
    data_dic = {}
    data_dic["price"] = data_list[0].text
    image = data_list[1].find("img")
    if image != None:
        if image["alt"] == "상승":
            data_dic["price_dif"] = data_list[1].text.strip("\n\t")
        else:
            data_dic["price_dif"] = "-" + data_list[1].text.strip("\n\t")
    else:
        data_dic["price_dif"] = 0
    data_dic["sell"] = data_list[2].text
    data_dic["buy"] = data_list[3].text
    data_dic["volume"] = data_list[4].text
    data_dic["variation"] = data_list[5].text
    
    return data_dic

In [6]:
time = []

prefix = "2018-02-14_focus/discussion_2018-02-14"
for hour in range(9,16):
    for minute in range(0, 6):
        time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ ":" + str(minute) + "0.json")

In [8]:
del time[41]
del time[40]

In [ ]:
df = pd.DataFrame(columns=[["y", "code", "name", "post_num_0", "unique_id_0", "click_0", "like_0",\
                            "dislike_0", "time_0","price_0","price_dif_0",\
                            "sell_0", "buy_0", "volume_0", "variation_0",
                            "post_num_1", "unique_id_1", "click_1", "like_1",\
                            "dislike_1", "time_1","price_1","price_dif_1",\
                            "sell_1", "buy_1", "volume_1", "variation_1",
                            "post_num_2", "unique_id_2", "click_2", "like_2",\
                            "dislike_2", "time_2","price_2","price_dif_2",\
                            "sell_2", "buy_2", "volume_2", "variation_2"]])

In [ ]:
for t in time:
    with open(t) as f:
        discussion = json.load(f)
    for d in discussion:
        if d["name"] in increase:
            row = pd.DataFrame(columns=["y", "code", "name", "post_num_0", "unique_id_0", "click_0", "like_0",\
                                "dislike_0", "time_0","price_0","price_dif_0",\
                                "sell_0", "buy_0", "volume_0", "variation_0",
                                "post_num_1", "unique_id_1", "click_1", "like_1",\
                                "dislike_1", "time_1","price_1","price_dif_1",\
                                "sell_1", "buy_1", "volume_1", "variation_1",
                                "post_num_2", "unique_id_2", "click_2", "like_2",\
                                "dislike_2", "time_2","price_2","price_dif_2",\
                                "sell_2", "buy_2", "volume_2", "variation_2"],\
                           data = [[np.nan, krx_code[d["name"]], d["name"], d["post_num"], d["unique_id"], d["click"], \
                                    d["like"], d["dislike"], d["time"], \
                                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,\
                                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,\
                                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]])
            df = df.append(row)

In [ ]:
df = df.reset_index()

In [ ]:
df.shape

In [ ]:
for idx, row in df.iterrows():
    timestamp = row["time_0"]
    t = re.sub('[ :-]', '', timestamp)
    
    d = scrape_price_history(row["code"], t)
    tpl = (d["price"], d["price_dif"], d["sell"], d["buy"], d["volume"], d["variation"])
    
    df["price_0"].iloc[idx] = tpl[0]
    df["price_dif_0"].iloc[idx] = tpl[1]
    df["sell_0"].iloc[idx] = tpl[2]
    df["buy_0"].iloc[idx] = tpl[3]
    df["volume_0"].iloc[idx] = tpl[4]
    df["variation_0"].iloc[idx] = tpl[5]
    
    print(idx)

In [ ]:
for idx, row in df.iterrows():
    if idx < len(df) - len(increase):
        df["y"].iloc[idx] = df["price_0"].iloc[idx + len(increase)]

In [ ]:
for idx, row in df.iterrows():
    if idx >= len(increase) * 2:
        df["post_num_2"].iloc[idx] = df["post_num_0"].iloc[idx - (len(increase) * 2)]
        df["unique_id_2"].iloc[idx] = df["unique_id_0"].iloc[idx - (len(increase) * 2)]
        df["click_2"].iloc[idx] = df["click_0"].iloc[idx - (len(increase) * 2)]
        df["like_2"].iloc[idx] = df["like_0"].iloc[idx - (len(increase) * 2)]
        df["dislike_2"].iloc[idx] = df["dislike_0"].iloc[idx - (len(increase) * 2)]
        df["time_2"].iloc[idx] = df["time_0"].iloc[idx - (len(increase) * 2)]
        
        df["price_2"].iloc[idx] = df["post_num_0"].iloc[idx - (len(increase) * 2)]
        df["price_dif_2"].iloc[idx] = df["price_dif_0"].iloc[idx - (len(increase) * 2)]
        df["sell_2"].iloc[idx] = df["sell_0"].iloc[idx - (len(increase) * 2)]
        df["buy_2"].iloc[idx] = df["buy_0"].iloc[idx - (len(increase) * 2)]
        df["volume_2"].iloc[idx] = df["volume_0"].iloc[idx - (len(increase) * 2)]
        df["variation_2"].iloc[idx] = df["variation_0"].iloc[idx - (len(increase) * 2)]

In [ ]:
for idx, row in df.iterrows():
    if idx >= len(increase) * 2:
        df["post_num_2"].iloc[idx] = df["post_num_0"].iloc[idx - (len(increase) * 2)]
        df["unique_id_2"].iloc[idx] = df["unique_id_0"].iloc[idx - (len(increase) * 2)]
        df["click_2"].iloc[idx] = df["click_0"].iloc[idx - (len(increase) * 2)]
        df["like_2"].iloc[idx] = df["like_0"].iloc[idx - (len(increase) * 2)]
        df["dislike_2"].iloc[idx] = df["dislike_0"].iloc[idx - (len(increase) * 2)]
        df["time_2"].iloc[idx] = df["time_0"].iloc[idx - (len(increase) * 2)]
        
        df["price_2"].iloc[idx] = df["post_num_0"].iloc[idx - (len(increase) * 2)]
        df["price_dif_2"].iloc[idx] = df["price_dif_0"].iloc[idx - (len(increase) * 2)]
        df["sell_2"].iloc[idx] = df["sell_0"].iloc[idx - (len(increase) * 2)]
        df["buy_2"].iloc[idx] = df["buy_0"].iloc[idx - (len(increase) * 2)]
        df["volume_2"].iloc[idx] = df["volume_0"].iloc[idx - (len(increase) * 2)]
        df["variation_2"].iloc[idx] = df["variation_0"].iloc[idx - (len(increase) * 2)]

In [ ]:
df.columns

In [ ]:
rv = []
rv.append(df.columns.tolist())
for row in df.iterrows():
    rv.append(row[1].tolist())

In [ ]:
#import json
#json = json.dumps(rv, ensure_ascii=False)
#f = open("2018-02-14_09:00_09:20_dataframe.json","w")
#f.write(json)
#f.close()